# 1. Set up working directory

In [1]:
!cd ~
!ls
!mkdir capstone_project
!cd capstone_project
!pwd

TP53_CDS.fa	  tp53_protein_seq.fa
capstone_project  tp53_reference_data_retrieval.ipynb
mkdir: cannot create directory ‘capstone_project’: File exists
/home/thalia/capstone_project


# 2. Retrieve TP3 Gene Information from Ensembl

In [2]:
!curl -s 'https://rest.ensembl.org/lookup/id/ENSG00000141510?expand=1' \
-H 'Content-type:application/json' | jq

{
  "seq_region_name": "17",
  "display_name": "TP53",
  "logic_name": "ensembl_havana_gene_homo_sapiens",
  "strand": -1,
  "end": 7687546,
  "db_type": "core",
  "species": "homo_sapiens",
  "biotype": "protein_coding",
  "Transcript": [
    {
      "gencode_primary": 0,
      "logic_name": "havana_homo_sapiens",
      "display_name": "TP53-203",
      "seq_region_name": "17",
      "strand": -1,
      "end": 7676594,
      "is_canonical": 0,
      "db_type": "core",
      "species": "homo_sapiens",
      "biotype": "protein_coding",
      "Parent": "ENSG00000141510",
      "start": 7661779,
      "assembly_name": "GRCh38",
      "Exon": [
        {
          "species": "homo_sapiens",
          "version": 1,
          "object_type": "Exon",
          "id": "ENSE00002337729",
          "db_type": "core",
          "end": 7676594,
          "strand": -1,
          "assembly_name": "GRCh38",
          "seq_region_name": "17",
          "start": 7676521
        },
        {
          "s

In [3]:
!curl -s 'https://rest.ensembl.org/lookup/id/ENSG00000141510?expand=1' \
-H 'Content-type:application/json' | jq -r '[.seq_region_name, .start, .end] | @tsv'

17	7661779	7687546


# 4. Download TP53 CDS from Ensembl (FASTA)

In [4]:
!curl -s 'https://rest.ensembl.org/sequence/id/ENST00000269305?type=cds' \
  -H 'Content-type:text/x-fasta' > TP53_CDS.fa

# 5. Translate to Protein Sequence with Biopython and Save to a FASTA file

In [5]:
from Bio import SeqIO

rec = SeqIO.read("TP53_CDS.fa", "fasta")
cds = rec.seq

# Check if sequence length is a multiple of 3
if len(cds) % 3 == 0:
    print("Sequence length is valid for translation.")
else:
    print("Warning: Sequence length not a multiple of 3. Trimming extra bases.")
    # Trim trailing bases that don't form a full codon
    cds = cds[:len(cds) - (len(cds) % 3)]

# Translate
protein = cds.translate(to_stop=True)
print(protein)

# Double-check to make sure this protein sequence is correct by cross-checking it with
# the protein sequence from UniProt - ID: P04637

# Save to FASTA file
with open("tp53_protein_seq.fa", "w") as f:
    f.write(">tp53_protein\n")
    f.write(str(protein) + "\n")

print("Protein sequence saved to tp53_protein_seq.fa")

Sequence length is valid for translation.
MEEPQSDPSVEPPLSQETFSDLWKLLPENNVLSPLPSQAMDDLMLSPDDIEQWFTEDPGPDEAPRMPEAAPPVAPAPAAPTPAAPAPAPSWPLSSSVPSQKTYQGSYGFRLGFLHSGTAKSVTCTYSPALNKMFCQLAKTCPVQLWVDSTPPPGTRVRAMAIYKQSQHMTEVVRRCPHHERCSDSDGLAPPQHLIRVEGNLRVEYLDDRNTFRHSVVVPYEPPEVGSDCTTIHYNYMCNSSCMGGMNRRPILTIITLEDSSGNLLGRNSFEVRVCACPGRDRRTEEENLRKKGEPHHELPPGSTKRALPNNTSSSPQPKKKPLDGEYFTLQIRGRERFEMFRELNEALELKDAQAGKEPGGSRAHSSHLKSKKGQSTSRHKKLMFKTEGPDSD
Protein sequence saved to tp53_protein_seq.fa


# 6. Check contents of new file

In [6]:
!cat tp53_protein_seq.fa | head

>tp53_protein
MEEPQSDPSVEPPLSQETFSDLWKLLPENNVLSPLPSQAMDDLMLSPDDIEQWFTEDPGPDEAPRMPEAAPPVAPAPAAPTPAAPAPAPSWPLSSSVPSQKTYQGSYGFRLGFLHSGTAKSVTCTYSPALNKMFCQLAKTCPVQLWVDSTPPPGTRVRAMAIYKQSQHMTEVVRRCPHHERCSDSDGLAPPQHLIRVEGNLRVEYLDDRNTFRHSVVVPYEPPEVGSDCTTIHYNYMCNSSCMGGMNRRPILTIITLEDSSGNLLGRNSFEVRVCACPGRDRRTEEENLRKKGEPHHELPPGSTKRALPNNTSSSPQPKKKPLDGEYFTLQIRGRERFEMFRELNEALELKDAQAGKEPGGSRAHSSHLKSKKGQSTSRHKKLMFKTEGPDSD


# Result: 

I now have a protein FASTA file ( tp53_protein_seq.fa ) derived from my TP53 cds FASTA which can now be used for 
downstream analyses like variant annotation and functional prediction.